In [31]:
import spinmob as s
import numpy as np
import matplotlib.pyplot as plt

import polars as pl
from scipy.special import factorial



In [2]:
from rho_calibration_custom import gain_factor
from T_sys_calibration_custom import Temp_system

In [32]:
P_COLD_PATH = "/home/slash/Desktop/Phys 258/Phys_258_Final_Project/Amateur_measurement_of_the_CMB/data/day2_cold_load_cal2"
P_HOT_PATH = "/home/slash/Desktop/Phys 258/Phys_258_Final_Project/Amateur_measurement_of_the_CMB/data/day2_hot_load_cal3"

T_hot = 276
T_cold = 150

p_cold = (np.fromfile(open(P_COLD_PATH), dtype=np.float32, count=1500))
p_hot = np.fromfile(open(P_HOT_PATH), dtype=np.float32, count=800)


G = gain_factor(T_hot, T_cold, p_hot, p_cold)
T_sys = Temp_system(T_hot, T_cold, p_hot, p_cold)

plt.plot(p_cold)
plt.plot(p_hot)
plt.show()


angles = [
    0.9,
    10.0,
    20.0,
    29.5,
    39.9 
    
]

angles = np.array(angles)
angles = np.radians(angles)

sec = lambda x: 1/np.cos(x)

angles = sec(angles)

P_ANGLE_PATHS = [
    "/home/slash/Desktop/Phys 258/Phys_258_Final_Project/Amateur_measurement_of_the_CMB/data/sky_measurement_1",
    "/home/slash/Desktop/Phys 258/Phys_258_Final_Project/Amateur_measurement_of_the_CMB/data/sky_measurement_2",
    "/home/slash/Desktop/Phys 258/Phys_258_Final_Project/Amateur_measurement_of_the_CMB/data/sky_measurement_3",
    "/home/slash/Desktop/Phys 258/Phys_258_Final_Project/Amateur_measurement_of_the_CMB/data/sky_measurement_4",
    "/home/slash/Desktop/Phys 258/Phys_258_Final_Project/Amateur_measurement_of_the_CMB/data/sky_measurement_5"
    
]

# csv_path = "../../../data/Sys_constants/T_atm.csv"

#loop over paths to average the power
power = []
power_std = []
power_a = []

power_raw = []

for i, P_PATH in enumerate(P_ANGLE_PATHS):
    power_file = G*np.fromfile(open(P_PATH), dtype=np.float32, count=1000)

    # Remove constant
   
    power.append(power_file.mean())
    power_std.append(power_file.std())
    power_a.append(power_file.std()/np.sqrt(len(power_file)))



    power_raw.append(power_file)

dfs = []
i = 0
for run in power_raw:
	dfs.append(pl.from_numpy(run, schema=str(i)))
	i += 1

df = pl.concat(dfs, how="horizontal")
df

  time_of_measurement          value  T_hot  T_cold
0    2023-04-04T12:51 -284612.485187    276     150
  time_of_measurement       value  T_hot  T_cold
0    2023-04-04T12:51 -859.027052    276     150


0,1,2,3,4
f32,f32,f32,f32,f32
-274.448181,-285.321472,-277.5896,-285.773041,-290.647736
-274.563568,-285.203247,-277.379883,-285.632721,-290.922699
-274.054382,-284.86557,-277.379181,-285.804413,-290.973877
-274.124298,-284.588135,-277.632416,-285.932678,-291.056274
-273.848724,-284.552612,-277.621185,-286.150665,-291.242035
-273.499847,-284.480438,-277.372742,-286.054443,-291.230591
-273.319763,-284.483734,-277.437683,-286.151154,-291.051605
-273.399414,-284.453705,-277.512329,-285.649963,-290.527283
-273.665222,-284.24585,-277.485992,-285.789276,-290.735748


In [33]:
colors = ['r','g','b','yellow', 'orange']
for run, color in zip(power_raw, colors):
	plt.plot(run, color=color)


In [6]:
means = []
stds = []
stderrs = []
for i in range(5):

	out = df.select(
		[
			pl.col(str(i)).filter(pl.col(str(i)) < 200)

		]
	)

	n = df.select(
		[
			pl.col(str(i)).filter(pl.col(str(i)) < 200).count()

		]
	)

	# out = df.select(
	# 	[
	# 		pl.when(pl.col("0") > 100).then(0).otherwise(pl.col("0"))
	# 	]
	# )

	plt.plot(out)
	means.append(out.mean())
	stds.append(out.std())
	stderrs.append(out.std()/n)
	# print(out)


means = pl.concat(means, how="horizontal")
stds = pl.concat(stds, how="horizontal")
stderrs = pl.concat(stderrs, how="horizontal")

print(means)
print(stds)
print(stderrs)


shape: (1, 5)
┌────────────┬────────────┬────────────┬────────────┬────────────┐
│ 0          ┆ 1          ┆ 2          ┆ 3          ┆ 4          │
│ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│ f32        ┆ f32        ┆ f32        ┆ f32        ┆ f32        │
╞════════════╪════════════╪════════════╪════════════╪════════════╡
│ 151.621628 ┆ 153.825577 ┆ 153.472946 ┆ 158.936737 ┆ 163.554306 │
└────────────┴────────────┴────────────┴────────────┴────────────┘
shape: (1, 5)
┌──────────┬──────────┬──────────┬──────────┬──────────┐
│ 0        ┆ 1        ┆ 2        ┆ 3        ┆ 4        │
│ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ f32      ┆ f32      ┆ f32      ┆ f32      ┆ f32      │
╞══════════╪══════════╪══════════╪══════════╪══════════╡
│ 1.964303 ┆ 2.236639 ┆ 0.549704 ┆ 0.256996 ┆ 1.681453 │
└──────────┴──────────┴──────────┴──────────┴──────────┘
shape: (1, 5)
┌──────────┬──────────┬──────────┬──────────┬──────────┐
│ 0        ┆ 1        ┆ 2        

In [7]:
f = s.data.fitter()
f.set_functions('m*x+c', 'm,c')
print(angles)
f.set_data(angles, means.to_numpy())

f.fit()

[1.00012338 1.01542661 1.06417777 1.14895554 1.30350029]



FITTER SETTINGS
  autoplot        True
  coarsen         [1]
  first_figure    0
  fpoints         [1000]
  plot_all_data   [False]
  plot_bg         [True]
  plot_errors     [True]
  plot_fit        [True]
  plot_guess      [True]
  plot_guess_zoom [False]
  scale_eydata    [1.0]
  silent          False
  style_bg        [{'marker': '', 'color': 'k', 'ls': '-'}]
  style_data      [{'marker': 'o', 'color': 'b', 'ls': '', 'mec': 'b'}]
  style_fit       [{'marker': '', 'color': 'r', 'ls': '-'}]
  style_guess     [{'marker': '', 'color': '0.25', 'ls': '-'}]
  subtract_bg     [False]
  xlabel          [None]
  xmax            [None]
  xmin            [None]
  xscale          ['linear']
  ylabel          [None]
  ymax            [None]
  ymin            [None]
  yscale          ['linear']

INPUT PARAMETERS (reduced chi^2 = 27844.36 +/- 0.82, 3.0 DOF)
  m = 1, vary=True min=-INF max=INF expr=None
  c = 1, vary=True min=-INF max=INF expr=None

FIT RESULTS (reduced chi^2 = 0.92 +/- 0.82, 3 DO

In [ ]:
for xe, ye in zip(angles, power_raw):
    plt.scatter([xe] * len(ye), ye, color='b', marker='.')

plt.scatter(angles, power, color='orange')
plt.errorbar(angles, power, power_a, color='orange', linestyle='')

<ErrorbarContainer object of 3 artists>

In [ ]:
s.plot.xy.data(angles, power, marker='o', linestyle='')


<AxesSubplot: title={'center': "\nPlot created Sat Apr  1 12:37:07 2023\ns.plot.xy.data(angles, power, marker='o', linestyle='')"}>